In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from modules import AthenaQueryExecutor, SqlRender
import pytz
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os

from extra.config import awsconfig
from extra.utils import (extract, load, utils)
from extra.constants import (trx_columns, data_id_pago, name_columns_masivo)
from extra import transforms

warnings.filterwarnings('ignore')
# plt.style.use("../../0. Matriz tarjeta - Proyecto base/dataviz_template/corporate.mplstyle")
# pd.set_option('display.float_format', '{:.2f}'.format)

In [10]:
# Obtener la fecha y hora actual
fecha_hora_actual = datetime.now()

# Configurar la zona horaria a 'America/Bogota'
zona_horaria_colombia = pytz.timezone('America/Bogota')
fecha_hora_colombia = fecha_hora_actual.astimezone(zona_horaria_colombia)

# fecha_result = fecha_hora_colombia - relativedelta(months=3)
# print(fecha_result.year)

string_filtro = ""

# creando el string_filtro con las fechas desde 3 meses atras.
for i in range(3):
    fecha_result = fecha_hora_colombia - relativedelta(months=i)
    string_filtro = string_filtro + "(year = '" + str(fecha_result.year) + "' AND month = '"+str(fecha_result.month).zfill(2)+"') OR "
string_filtro = string_filtro[:-4]
print(string_filtro)

(year = '2025' AND month = '01') OR (year = '2024' AND month = '12') OR (year = '2024' AND month = '11')


In [11]:
awsconfig.aws_config_profile('NEQUI-DataBusinessAnalyst-Role-044879804046')

Perfil AWS configurado: NEQUI-DataBusinessAnalyst-Role-044879804046


In [12]:
query_malla_prot_crisis = SqlRender.render_template(name_template="datos_tickets_all.sql",database="co_delfos_servicio_raw_pdn_rl",
                                                    table="co_zendesk_flatten_tickets", filtro_fecha=string_filtro)

print(query_malla_prot_crisis)

df_consulta = AthenaQueryExecutor.execute_query(query=query_malla_prot_crisis, database_athena="co_delfos_servicio_raw_pdn_rl", 
                                                workgroup_athena="delfos")


-- (
-- select  ticket_id AS "ID_del_ticket"
--         , fields_payload['numero_de_documento'] AS "Numero_de_documento"
--         , fields_payload['modalidad_de_fraude'] AS "Modalidad_de_fraude"
--         , fields_payload['tipo_de_incidente'] AS "Tipo_de_incidente"
--         , created_at AS "Ticket_creado_Fecha"
--         , type AS "Tipo_de_ticket"
--         , fields_payload['valor_del_fraude'] AS "Valor_del_fraude"
-- from co_delfos_servicio_raw_pdn_rl.co_zendesk_flatten_tickets
-- where   true
--         and ((year = '2025' AND month = '01') OR (year = '2024' AND month = '12') OR (year = '2024' AND month = '11'))
--         and ticket_form_label = 'gestion_de_fraude'
--         and type IN ('incident') --Tipo ticket
--         --and fields_payload['tipo_de_incidente'] in ('reclamo_smart','queja_smart','solicitud_smart')
--         -- and fields_payload['tipo_de_incidente'] in ('reclamo_smart', 'solicitud_smart')

--         and ((fields_payload['tipo_de_incidente'] in ('reclamo

In [13]:
cols = ['ID del ticket', 'Número de documento', 'Modalidad', 'Tipo de incidente', 
        'Ticket creado - Fecha', 'Tipo de ticket', 'Valor del fraude']

rename_columns = {  'ID_del_ticket':'ID del ticket',
                    'Numero_de_documento':'Número de documento',
                    'Modalidad_de_fraude':'Modalidad',
                    'Tipo_de_incidente':'Tipo de incidente',
                    'Ticket_creado_Fecha':'Ticket creado - Fecha',
                    'Tipo_de_ticket':'Tipo de ticket',
                    'Valor_del_fraude':'Valor del fraude'
                }

# rename_columns = {  'Modalidad de fraude':'Modalidad'}

df_consulta = df_consulta.rename(columns=rename_columns)

# Regla para volver cero el valor de fraude si el tipo de ticket es incidente.
df_consulta.loc[df_consulta['Tipo de ticket'] == 'question', 'Valor del fraude'] = '0'

df = df_consulta.copy()


# Concateno todas las modalidades
df['Ticket creado - Fecha'] = pd.to_datetime(df['Ticket creado - Fecha'])
df['Anio-Mes'] = df['Ticket creado - Fecha'].dt.strftime('%Y-%m')

print(df.shape)

Total de solicitudes: 5396
Total de reclamos: 48517
(51705, 8)


In [14]:
df.groupby('Modalidad')['Modalidad'].count()

Modalidad
compras_no_reconocidas_con_tarjeta_débito_modalidad     6033
estafa_modalidad                                         227
listas_restrictivas_bloqueos                            1029
receptores_de_fraude_modalidad                          1993
suplantación_de_identidad_modalidad                     3300
vulneración_de_cuentas_modalidad                       39123
Name: Modalidad, dtype: int64

## Analisis de registros que se eliminarian

In [15]:
# # Pruebas de condicional
# print(f"Tamaño inicial de la muestra: {df.shape[0]}")
# df['Valor del fraude'] = df['Valor del fraude'].astype('Float64')

# percentil = 99
# umbral = np.percentile(df['Valor del fraude'].dropna(), percentil)

# print(f"Percentil {percentil} de valores de fraude : {round(umbral, 2)}")
# print(f"Se pierden un {round(df[((df['Valor del fraude'] > umbral) & (df['Valor del fraude'].notna()) & (df['Valor del fraude'] != 0))].shape[0]/df.shape[0]*100, 2)}% de los datos")

# df = df[~((df['Valor del fraude'] > umbral) & (df['Valor del fraude'].notna()) & (df['Valor del fraude'] != 0))]
# print(f"Tamaño de la muestra limpia: {df.shape[0]}\n")

<class 'pandas.core.frame.DataFrame'>
Index: 51705 entries, 0 to 53912
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del ticket          51705 non-null  Int64         
 1   Número de documento    50506 non-null  string        
 2   Modalidad              51705 non-null  string        
 3   Tipo de incidente      51705 non-null  string        
 4   Ticket creado - Fecha  51705 non-null  datetime64[ns]
 5   Tipo de ticket         51705 non-null  string        
 6   Valor del fraude       45041 non-null  string        
 7   Anio-Mes               51705 non-null  object        
dtypes: Int64(1), datetime64[ns](1), object(1), string(5)
memory usage: 3.6+ MB


In [67]:
# ## Agrupación por cada evento (Número de documento, Modalidad de reclamo Gestión de Fraude y Ticket creado - Fecha )
# df = df.groupby(['Número de documento', 'Tipo de incidente', 'Modalidad', 'Ticket creado - Fecha']).agg({'ID del ticket': 'count', 'Valor del fraude': 'sum'}).sort_values('ID del ticket').reset_index()
# df['Anio-Mes'] = df['Ticket creado - Fecha'].dt.strftime('%Y-%m')
# df['Número de documento'] = df['Número de documento'].astype(str)
# print(f"Tamaño de la muestra agrupada: {df.shape[0]}\n")
# df.to_parquet("./data/processed/data_eventos_fraude.parquet", engine='pyarrow')

# df.to_excel("./data/processed/data_eventos_fraude_2.xlsx")
# df


In [16]:
## Fix: sin errores de ejecución.

print(f"Tamaño inicial de la muestra: {df.shape[0]}")
df = df.dropna(subset=['Número de documento', 'Tipo de incidente', 'Modalidad', 'Ticket creado - Fecha'])
print(f"Tamaño de la muestra eliminando campos vacíos: {df.shape[0]}\n")

df['Valor del fraude'] = df['Valor del fraude'].astype('Float64')

percentil = 99
umbral = np.percentile(df['Valor del fraude'].dropna(), percentil)

print(f"Percentil {percentil} de valores de fraude : {round(umbral, 2)}")
print(f"Se pierden un {round(df[df['Valor del fraude']>umbral].shape[0]/df.shape[0]*100, 2)}% de los datos")
df = df[~(df['Valor del fraude']>umbral)]
print(f"Tamaño de la muestra limpia: {df.shape[0]}\n")

## Agrupación por cada evento (Número de documento, Modalidad de reclamo Gestión de Fraude y Ticket creado - Fecha )
df = df.groupby(['Número de documento', 'Tipo de incidente', 'Modalidad', 'Ticket creado - Fecha']).agg({'ID del ticket': 'count', 'Valor del fraude': 'sum'}).sort_values('ID del ticket').reset_index()
df['Anio-Mes'] = df['Ticket creado - Fecha'].dt.strftime('%Y-%m')
df['Número de documento'] = df['Número de documento'].astype(str)
print(f"Tamaño de la muestra agrupada: {df.shape[0]}\n")
df.to_parquet("./data/processed/data_eventos_fraude.parquet", engine='pyarrow')

df

Tamaño inicial de la muestra: 51705
Tamaño de la muestra eliminando campos vacíos: 50506

Percentil 99 de valores de fraude : 3700000.0
Se pierden un 0.87% de los datos
Tamaño de la muestra limpia: 44520

Tamaño de la muestra agrupada: 44520



,Número de documento,Tipo de incidente,Modalidad,Ticket creado - Fecha,ID del ticket,Valor del fraude,Anio-Mes
0,1000000656,reclamo_smart,compras_no_reconocidas_con_tarjeta_débito_moda...,2024-11-20 22:43:06,1,93000.0,2024-11
1,1233899404,reclamo_smart,vulneración_de_cuentas_modalidad,2025-01-06 13:45:43,1,1100000.0,2025-01
2,1233899748,reclamo_smart,vulneración_de_cuentas_modalidad,2024-11-02 22:56:35,1,20000.0,2024-11
3,1233899786,reclamo_smart,vulneración_de_cuentas_modalidad,2024-11-10 18:24:50,1,40000.0,2024-11
4,1233899801,reclamo_smart,compras_no_reconocidas_con_tarjeta_débito_moda...,2024-12-15 11:19:36,1,66535.0,2024-12
...,...,...,...,...,...,...,...
44515,1053863838,reclamo_smart,vulneración_de_cuentas_modalidad,2024-11-01 10:21:27,1,130000.0,2024-11
44516,1053864517,reclamo_smart,vulneración_de_cuentas_modalidad,2024-12-06 21:48:30,1,709056.0,2024-12
44517,1053864971,reclamo_smart,vulneración_de_cuentas_modalidad,2025-01-25 11:39:24,1,1597492.0,2025-01
44518,1053859463,reclamo_smart,vulneración_de_cuentas_modalidad,2024-12-04 15:06:59,1,50000.0,2024-12
